In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import os
from tqdm.notebook import tqdm
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
!unzip -qq /content/drive/'My Drive'/Code/tomatoes.zip

In [4]:
dataset_path = '/content/PlantVillage'

class_name = []
images = []
labels = []
for index, folder in enumerate(os.listdir(dataset_path)):
  class_name.append(folder)
  image_path = os.path.join(dataset_path, folder)
  # print(image_path, len(os.listdir(image_path)))
  for image in os.listdir(image_path):
    labels.append(index)
    images.append(os.path.join(dataset_path, folder, image))

X = []
for index, path in enumerate(tqdm(images)):
  try:
    X.append(cv2.resize(cv2.imread(path),dsize=(256,256)))
  except:
    labels.pop(index)
X = np.array(X)
y = np.array(labels)
Y = to_categorical(y, num_classes=len(class_name))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,stratify = Y, shuffle=True, random_state=100)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify = y_test, shuffle=True, random_state=100)
print(X_train.shape, X_test.shape,X_val.shape)
print(y_train.shape, y_test.shape,y_val.shape)

(10946, 256, 256, 3) (2346, 256, 256, 3) (2346, 256, 256, 3)
(10946, 9) (2346, 9) (2346, 9)


In [6]:
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.1,
    rescale=1./255,
	width_shift_range=0.1,
    height_shift_range=0.1,
	horizontal_flip=True,
    brightness_range=[0.2,1.5], fill_mode="nearest")

aug_val = ImageDataGenerator(rescale=1./255)

In [ ]:
from keras.optimizers import Adam
def get_model():
    model_effnet_conv = EfficientNetB7(include_top = False, weights = 'imagenet')
    # freeze all layers in the base model
    for layer in model_effnet_conv.layers:
        layer.trainable = False

    input = Input(shape=(256, 256, 3), name='image_input')
    output_effnet_conv = model_effnet_conv(input)

    x = Flatten(name='flatten')(output_effnet_conv)
    x = Dense(128, activation='relu', name='fc1')(x)
    x = Dropout(0.5)(x)
    x = GlobalAveragePooling2D()(output_effnet_conv)
    x = Dense(64, activation='relu', name='fc2')(x)
    x = Dropout(0.5)(x)
    x = Dense(9, activation='softmax', name='predictions')(x)

    # Compile
    model = Model(input,x)
    
    model.compile(optimizer='Adam',loss = 'categorical_crossentropy',metrics=['categorical_accuracy'])
    
    return model

effnetmodel = get_model()
filepath="/content/weights-{epoch:02d}-{val_categorical_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

EfficientNetB7_NS = effnetmodel.fit(X_train, y_train, batch_size=64,
                              epochs=40, steps_per_epoch=len(X_train)//64,
                              validation_data=aug.flow(X_val,y_val,
                              batch_size=64), callbacks = callbacks_list)
effnetmodel.save("effi_net.h5")

Epoch 1/40
171/171 [==============================] - ETA: 0s - loss: 1.1371 - categorical_accuracy: 0.6208
Epoch 00001: val_categorical_accuracy improved from -inf to 0.10145, saving model to /content/weights-01-0.10.hdf5
171/171 [==============================] - 134s 782ms/step - loss: 1.1371 - categorical_accuracy: 0.6208 - val_loss: 2.5283 - val_categorical_accuracy: 0.1014
Epoch 2/40
171/171 [==============================] - ETA: 0s - loss: 0.7112 - categorical_accuracy: 0.7621
Epoch 00002: val_categorical_accuracy did not improve from 0.10145
171/171 [==============================] - 124s 727ms/step - loss: 0.7112 - categorical_accuracy: 0.7621 - val_loss: 3.2721 - val_categorical_accuracy: 0.1014
Epoch 3/40
171/171 [==============================] - ETA: 0s - loss: 0.6177 - categorical_accuracy: 0.7956
Epoch 00003: val_categorical_accuracy did not improve from 0.10145
171/171 [==============================] - 124s 724ms/step - loss: 0.6177 - categorical_accuracy: 0.7956 - va

In [ ]:
from keras.models import load_model
model = load_model('/content/effi_net.h5')
print("[INFO] Calculating model accuracy")
scores = model.evaluate(X_test, y_test)
#scores = vggmodell.evaluate(X_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")